# PVSC 2021 Paper: by STATES
### Electrification Futures BASE Installations, 
### with Module Composition Scenarios for 'TODAY' and 'Bifacial Projection';
### Just PV ICE Reliability Approach, no Bifaciality Factor Assumptions


SUMMARY OF OBJECTS:
r1 - Today. scenarios: STATES
r2 - Bifacial Projection. scenarios: STATES



In [1]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'PV_ICE' / 'TEMP' / 'PVSC')

print ("Your simulation will be stored in %s" % testfolder)

Your simulation will be stored in C:\Users\sayala\Documents\GitHub\CircularEconomy-MassFlowCalculator\PV_ICE\TEMP\PVSC


In [2]:
if not os.path.exists(testfolder):
    os.makedirs(testfolder)

In [3]:
MATERIALS = ['glass','silver','silicon', 'copper','aluminium_frames']
MATERIAL = MATERIALS[0]

MODULEBASELINE = r'..\..\baselines\ElectrificationFutures_2021\EF-CapacityByState-basecase.csv'
## COMMENTED OUT BECAUSE WE ARE NOT DOING THE "HIGH ELECTRIFICATION SCENARIO¶
#MODULEBASELINE_High = r'..\..\baselines\LiteratureProjections\EF-CapacityByState-LowREHighElec.csv'
MODULEBASELINE_OtherParams = r'..\..\baselines\ElectrificationFutures_2021\baseline_modules_US_NREL_Electrification_Futures_2021_basecase.csv'


In [4]:
import PV_ICE
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [5]:
PV_ICE.__version__

'0+untagged.416.g1b87254.dirty'

In [6]:
plt.rcParams.update({'font.size': 22})
plt.rcParams['figure.figsize'] = (12, 5)

# Loading Module Baseline. 
## Will be used later to populate all the columsn otehr than 'new_Installed_Capacity_[MW]' which will be supplied by the REEDS model

In [7]:
r1 = PV_ICE.Simulation(name='Simulation1', path=testfolder)
r1.createScenario(name='US', file=MODULEBASELINE_OtherParams)
baseline = r1.scenario['US'].data
baseline = baseline.drop(columns=['new_Installed_Capacity_[MW]'])
baseline.set_index('year', inplace=True)
baseline.index = pd.PeriodIndex(baseline.index, freq='A')  # A -- Annual
baseline.head()

path = C:\Users\sayala\Documents\GitHub\CircularEconomy-MassFlowCalculator\PV_ICE\TEMP\PVSC


,mod_eff,mod_reliability_t50,mod_reliability_t90,mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,mod_Repair,mod_MerchantTail,mod_Reuse
year,,,,,,,,,,,
1995,12.50,16.0,21.0,0.55,10.0,98.0,15.0,40.0,0.0,0.0,0.0
1996,12.70,16.0,21.0,0.55,10.0,98.0,15.0,40.0,0.0,0.0,0.0
1997,12.88,19.0,23.0,0.55,15.0,98.0,15.0,40.0,0.0,0.0,0.0
1998,13.06,19.0,23.0,0.55,15.0,98.0,15.0,40.0,0.0,0.0,0.0
1999,13.24,19.0,23.0,0.55,15.0,98.0,15.0,40.0,0.0,0.0,0.0


## Important: STATE DATA STARTS ON 2018, so dropping columns for making the baselines

In [8]:
baseline = baseline.iloc[23:]

## MERGE Baseline with States Instals

In [9]:
df = pd.read_csv(MODULEBASELINE)
df.set_index(['Type','State','year'], inplace=True)
df.head()

New Installs
Type State year              
PV   AL    2018    205.018300
           2019      2.210001
           2020      2.210001
           2021      4.017780
           2022      4.017780

In [13]:
for ii in range (len(df.unstack(level=2))):   
    STATE = df.unstack(level=2).iloc[ii].name[1]
    SCEN = df.unstack(level=2).iloc[ii].name[0]
    SCEN=SCEN.replace('+', '_')
    
    subtestfolder =  os.path.join(testfolder, 'STATES')
    if not os.path.exists(subtestfolder):
        os.makedirs(subtestfolder)
    
    filetitle = 'base_'+SCEN+'_'+STATE +'.csv'
    filetitle = os.path.join(subtestfolder, filetitle)
    A = df.unstack(level=2).iloc[ii]
    A = A.droplevel(level=0)
    A.name = 'new_Installed_Capacity_[MW]'
    A = pd.DataFrame(A)
    A.index=pd.PeriodIndex(A.index, freq='A')
    A = pd.DataFrame(A)
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] # 0.85 marketshares['Si'] already included
    # Add other columns
    A = pd.concat([A, baseline.reindex(A.index)], axis=1)
    
    
    header = "year,new_Installed_Capacity_[MW],mod_eff,mod_reliability_t50,mod_reliability_t90,"\
    "mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,"\
    "mod_Repair,mod_MerchantTail,mod_Reuse\n"\
    "year,MW,%,years,years,%,years,%,%,%,%,%\n"

    with open(filetitle, 'w', newline='') as ict:
    # Write the header lines, including the index variable for
    # the last one if you're letting Pandas produce that for you.
    # (see above).
        for line in header:
            ict.write(line)

        #    savedata.to_csv(ict, index=False)
        A.to_csv(ict, header=False)

#### COMMENTED OUT BECAUSE WE ARE NOT DOING THE "HIGH ELECTRIFICATION SCENARIO

In [14]:
## COMMENTED OUT BECAUSE WE ARE NOT DOING THE "HIGH ELECTRIFICATION SCENARIO
"""
df = pd.read_csv(MODULEBASELINE_High)
df.set_index(['Type','State','year'], inplace=True)
df.head()

for ii in range (len(df.unstack(level=2))):   
    STATE = df.unstack(level=2).iloc[ii].name[1]
    SCEN = df.unstack(level=2).iloc[ii].name[0]
    SCEN=SCEN.replace('+', '_')
    filetitle = 'LowREHighElec_'+SCEN+'_'+STATE +'.csv'
    filetitle = os.path.join(testfolder, 'STATEs', filetitle)
    A = df.unstack(level=2).iloc[ii]
    A = A.droplevel(level=0)
    A.name = 'new_Installed_Capacity_[MW]'
    A = pd.DataFrame(A)
    A.index=pd.PeriodIndex(A.index, freq='A')
    A = pd.DataFrame(A)
    A['new_Installed_Capacity_[MW]'] = A['new_Installed_Capacity_[MW]'] # 0.85 marketshares['Si'] already included

    # Add other columns
    A = pd.concat([A, baseline.reindex(A.index)], axis=1)
    
    
    header = "year,new_Installed_Capacity_[MW],mod_eff,mod_reliability_t50,mod_reliability_t90,"\
    "mod_degradation,mod_lifetime,mod_MFG_eff,mod_EOL_collection_eff,mod_EOL_collected_recycled,"\
    "mod_Repowering,mod_Repairing\n"\
    "year,MW,%,years,years,%,years,%,%,%,%,%\n"

    with open(filetitle, 'w', newline='') as ict:
    # Write the header lines, including the index variable for
    # the last one if you're letting Pandas produce that for you.
    # (see above).
        for line in header:
            ict.write(line)

        #    savedata.to_csv(ict, index=False)
        A.to_csv(ict, header=False)
"""
pass

# DO SIMULATIONS

In [15]:
#### Not really doing anything with Sims 'LowReHighElec' at the moment
Sims=['base','LowREHighElec']

In [16]:
baselineinstalls = os.path.join(testfolder, 'STATEs')
onlyfiles = [f for f in os.listdir(baselineinstalls)]

In [17]:
sim1 = [f for f in onlyfiles if f.startswith(Sims[0])]
## Not really doing anything with Sims 'LowReHighElec' at the moment
#sim2 = [f for f in onlyfiles if f.startswith(Sims[1])]
sim1[2]

'base_PV_AZ.csv'

#### Get List of States

In [18]:
STATEs = [i.split('_', 4)[2] for i in sim1]
STATEs = [i.split('.', 1)[0] for i in STATEs]

In [19]:
MATERIALS = ['glass','aluminium_frames','silver','silicon', 'copper']

In [20]:
MATERIALBASELINE_GLASS_TODAY = r'..\..\baselines\PVSC_2021\baseline_material_glass_hold2020.csv'
MATERIALBASELINE_ALFrames_TODAY = r'..\..\baselines\PVSC_2021\baseline_material_aluminium_frames_hold2020.csv'
MATERIALBASELINE_GLASS_BIFACIALPROJECTION = r'..\..\baselines\PVSC_2021\baseline_material_glass_bifacialTrend.csv'
MATERIALBASELINE_ALFrames_BIFACIALPROJECTION = r'..\..\baselines\PVSC_2021\baseline_material_aluminium_frames_bifacialTrend.csv'


# TODAY
r1 = PV_ICE.Simulation(name='Today', path=testfolder)

for jj in range (0, len(STATEs)): 
    filetitle = sim1[jj]
    filetitle = os.path.join(subtestfolder, filetitle) 
    scen = STATEs[jj]
    r1.createScenario(name=scen, file=filetitle)
    r1.scenario[scen].addMaterial('glass', file=MATERIALBASELINE_GLASS_TODAY)
    r1.scenario[scen].material['glass'].materialdata = r1.scenario[scen].material['glass'].materialdata.iloc[23:].reset_index(drop=True)
    
    r1.scenario[scen].material['glass'].materialdata = r1.scenario[scen].material['glass'].materialdata
    r1.scenario[scen].addMaterial('aluminium_frames', file=MATERIALBASELINE_ALFrames_TODAY)
    r1.scenario[scen].material['aluminium_frames'].materialdata = r1.scenario[scen].material['aluminium_frames'].materialdata.iloc[23:].reset_index(drop=True)
    
    for mat in range (2, len(MATERIALS)):
        MATERIALBASELINE=r'..\..\baselines\baseline_material_'+MATERIALS[mat]+'.csv'
        r1.scenario[scen].addMaterial(MATERIALS[mat], file=MATERIALBASELINE)
        r1.scenario[scen].material[MATERIALS[mat]].materialdata = r1.scenario[scen].material[MATERIALS[mat]].materialdata.iloc[23:].reset_index(drop=True)

# Bifacial Projection
r2 = PV_ICE.Simulation(name='Bifacial', path=testfolder)

for jj in range (0, len(STATEs)): 
    filetitle = sim1[jj]
    filetitle = os.path.join(subtestfolder, filetitle) 
    scen = STATEs[jj]
    r2.createScenario(name=scen, file=filetitle)
    r2.scenario[scen].addMaterial('glass', file=MATERIALBASELINE_GLASS_BIFACIALPROJECTION)
    r2.scenario[scen].material['glass'].materialdata = r2.scenario[scen].material['glass'].materialdata.iloc[23:].reset_index(drop=True)
    
    r2.scenario[scen].material['glass'].materialdata = r2.scenario[scen].material['glass'].materialdata
    r2.scenario[scen].addMaterial('aluminium_frames', file=MATERIALBASELINE_ALFrames_BIFACIALPROJECTION)
    r2.scenario[scen].material['aluminium_frames'].materialdata = r2.scenario[scen].material['aluminium_frames'].materialdata.iloc[23:].reset_index(drop=True)
    
    for mat in range (2, len(MATERIALS)):
        MATERIALBASELINE=r'..\..\baselines\baseline_material_'+MATERIALS[mat]+'.csv'
        r2.scenario[scen].addMaterial(MATERIALS[mat], file=MATERIALBASELINE)
        r2.scenario[scen].material[MATERIALS[mat]].materialdata = r2.scenario[scen].material[MATERIALS[mat]].materialdata.iloc[23:].reset_index(drop=True)


path = C:\Users\sayala\Documents\GitHub\CircularEconomy-MassFlowCalculator\PV_ICE\TEMP\PVSC
path = C:\Users\sayala\Documents\GitHub\CircularEconomy-MassFlowCalculator\PV_ICE\TEMP\PVSC


In [21]:
r1.calculateMassFlow()
r2.calculateMassFlow()


Working on Scenario:  AL
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  aluminium_frames
==> Working on Material :  silver
==> Working on Material :  silicon
==> Working on Material :  copper
Working on Scenario:  AR
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  aluminium_frames
==> Working on Material :  silver
==> Working on Material :  silicon
==> Working on Material :  copper
Working on Scenario:  AZ
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  aluminium_frames
==> Working on Material :  silver
==> Working on Material :  silicon
==> Working on Material :  copper
Working on Scenario:  CA
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  aluminium_frames
==> Working on Material

==> Working on Material :  silver
==> Working on Material :  silicon
==> Working on Material :  copper
Working on Scenario:  NY
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  aluminium_frames
==> Working on Material :  silver
==> Working on Material :  silicon
==> Working on Material :  copper
Working on Scenario:  OH
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  aluminium_frames
==> Working on Material :  silver
==> Working on Material :  silicon
==> Working on Material :  copper
Working on Scenario:  OK
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  aluminium_frames
==> Working on Material :  silver
==> Working on Material :  silicon
==> Working on Material :  copper
Working on Scenario:  OR
********************
Finished Area+Power Generation Calculatio

Working on Scenario:  KS
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  aluminium_frames
==> Working on Material :  silver
==> Working on Material :  silicon
==> Working on Material :  copper
Working on Scenario:  KY
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  aluminium_frames
==> Working on Material :  silver
==> Working on Material :  silicon
==> Working on Material :  copper
Working on Scenario:  LA
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  aluminium_frames
==> Working on Material :  silver
==> Working on Material :  silicon
==> Working on Material :  copper
Working on Scenario:  MA
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  aluminium_frames
==> Working on Material

==> Working on Material :  aluminium_frames
==> Working on Material :  silver
==> Working on Material :  silicon
==> Working on Material :  copper
Working on Scenario:  WA
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  aluminium_frames
==> Working on Material :  silver
==> Working on Material :  silicon
==> Working on Material :  copper
Working on Scenario:  WI
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  aluminium_frames
==> Working on Material :  silver
==> Working on Material :  silicon
==> Working on Material :  copper
Working on Scenario:  WV
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  aluminium_frames
==> Working on Material :  silver
==> Working on Material :  silicon
==> Working on Material :  copper
Working on Scenario:  WY
******************

# OPEN EI

In [22]:
SFScenarios = [r1, r2]
SFScenarios[0].name

'Today'

In [23]:
# WORK ON THIS FOIR OPENEI

keyw=['mat_Virgin_Stock','mat_Total_EOL_Landfilled','mat_Total_MFG_Landfilled', 'mat_Total_Landfilled', 
      'new_Installed_Capacity_[MW]','Installed_Capacity_[W]']
keywprint = ['VirginMaterialDemand','EOLMaterial', 'ManufacturingScrap','ManufacturingScrapAndEOLMaterial',
             'NewInstalledCapacity','InstalledCapacity'] 
keywunits = ['MetricTonnes', 'MetricTonnes', 'MetricTonnes', 'MetricTonnes', 
            'MW','MW']
keywdcumneed = [True,True,True,True,
                True,False]
keywdlevel = ['material','material','material','material',
             'module','module']
keywscale = [1000000, 1000000, 1000000, 1000000,
            1,1e6]
materials = ['glass', 'silicon', 'silver', 'copper', 'aluminum']
materials = ['glass','aluminium_frames','silver','silicon', 'copper']

# Loop over SF Scenarios

scenariolist = pd.DataFrame()
for kk in range(0, len(SFScenarios)):
    # Loop over Materials
    
    for zz in range (0, len(STATEs)):

        foo = pd.DataFrame()
        for jj in range (0, len(keyw)):

            if keywdlevel[jj] == 'material':
                for ii in range (0, len(materials)):    
                    scen = STATEs[zz]
                    sentit = '@value|'+keywprint[jj]+'|'+materials[ii].capitalize() +'#'+keywunits[jj]
                    foo[sentit] = SFScenarios[kk].scenario[scen].material[materials[ii]].materialdata[keyw[jj]]/keywscale[jj] 
            
                if keywdcumneed[jj]:
                    for ii in range (0, len(materials)):    
                        sentit = '@value|Cumulative'+keywprint[jj]+'|'+materials[ii].capitalize() +'#'+keywunits[jj]
                        foo[sentit] = SFScenarios[kk].scenario[scen].material[materials[ii]].materialdata[keyw[jj]].cumsum()/keywscale[jj] 

            else:
                sentit = '@value|'+keywprint[jj]+'|'+'PV' +'#'+keywunits[jj]
                #sentit = '@value|'+keywprint[jj]+'#'+keywunits[jj]
                foo[sentit] = SFScenarios[kk].scenario[scen].data[keyw[jj]]/keywscale[jj] 

                if keywdcumneed[jj]:
                    sentit = '@value|Cumulative'+keywprint[jj]+'|'+'PV' +'#'+keywunits[jj]
                    foo[sentit] = SFScenarios[kk].scenario[scen].data[keyw[jj]].cumsum()/keywscale[jj] 
                  

        foo['@states'] = STATEs[zz]
        foo['@scenario|Module Composition Scenario'] = SFScenarios[kk].name 
#        foo['@filter|Install'] = Type[zz]
        foo['@timeseries|Year'] = SFScenarios[kk].scenario[scen].data.year

        scenariolist = scenariolist.append(foo)   

cols = [scenariolist.columns[-1]] + [col for col in scenariolist if col != scenariolist.columns[-1]]
scenariolist = scenariolist[cols]
cols = [scenariolist.columns[-1]] + [col for col in scenariolist if col != scenariolist.columns[-1]]
scenariolist = scenariolist[cols]
cols = [scenariolist.columns[-1]] + [col for col in scenariolist if col != scenariolist.columns[-1]]
scenariolist = scenariolist[cols]
#cols = [scenariolist.columns[-1]] + [col for col in scenariolist if col != scenariolist.columns[-1]]
#scenariolist = scenariolist[cols]
#scenariolist = scenariolist/1000000 # Converting to Metric Tons
#scenariolist = scenariolist.applymap(lambda x: round(x, N - int(np.floor(np.log10(abs(x))))))
#scenariolist = scenariolist.applymap(lambda x: int(x))
scenariolist.to_csv('OPEN EI 1 - STATE - PVSC 2021 PV ICE Today and Bifacial Projection All columns.csv', index=False)

print("Done")

Done


In [24]:
scenariolist.head(5)

,@states,@scenario|Module Composition Scenario,@timeseries|Year,@value|VirginMaterialDemand|Glass#MetricTonnes,@value|VirginMaterialDemand|Aluminium_frames#MetricTonnes,@value|VirginMaterialDemand|Silver#MetricTonnes,@value|VirginMaterialDemand|Silicon#MetricTonnes,@value|VirginMaterialDemand|Copper#MetricTonnes,@value|CumulativeVirginMaterialDemand|Glass#MetricTonnes,@value|CumulativeVirginMaterialDemand|Aluminium_frames#MetricTonnes,...,@value|ManufacturingScrapAndEOLMaterial|Silicon#MetricTonnes,@value|ManufacturingScrapAndEOLMaterial|Copper#MetricTonnes,@value|CumulativeManufacturingScrapAndEOLMaterial|Glass#MetricTonnes,@value|CumulativeManufacturingScrapAndEOLMaterial|Aluminium_frames#MetricTonnes,@value|CumulativeManufacturingScrapAndEOLMaterial|Silver#MetricTonnes,@value|CumulativeManufacturingScrapAndEOLMaterial|Silicon#MetricTonnes,@value|CumulativeManufacturingScrapAndEOLMaterial|Copper#MetricTonnes,@value|NewInstalledCapacity|PV#MW,@value|CumulativeNewInstalledCapacity|PV#MW,@value|InstalledCapacity|PV#MW
0,AL,Today,2018,9864.362332,1535.850331,5.928266,734.473996,9.084235,9864.362332,1535.850331,...,221.349653,0.044006,338.936866,18.640630,0.103931,221.349653,0.044006,205.018300,205.018300,206.132530
1,AL,Today,2019,104.826957,15.655958,0.058876,7.259433,0.103995,9969.189288,1551.506289,...,2.104500,0.000504,342.415305,18.830647,0.104963,223.454153,0.044510,2.210001,207.228300,207.239811
2,AL,Today,2020,103.414636,14.842270,0.048383,6.342070,0.098162,10072.603925,1566.348559,...,1.755881,0.000476,345.725184,19.010792,0.105812,225.210034,0.044986,2.210001,209.438301,208.834158
3,AL,Today,2021,181.668781,26.153814,0.079240,10.891912,0.173916,10254.272706,1592.502373,...,3.062201,0.000843,351.219398,19.328305,0.107202,228.272235,0.045829,4.017780,213.456081,212.240403
4,AL,Today,2022,177.325575,25.580821,0.071759,10.378806,0.171027,10431.598281,1618.083195,...,2.966397,0.000832,356.377279,19.639466,0.108462,231.238632,0.046661,4.017780,217.473861,215.628637


In [ ]:
#scenariolist.keys()

### Save REDUCED CSV
###### Just Capacity & Module EOL Waste Columns

In [25]:
csvtitle = 'OPEN EI 2 - STATE - PVSC 2021 PV ICE Today and Bifacial Projection Selected Columns.csv'
colNames = scenariolist.columns[scenariolist.columns.str.contains('CumulativeEOLMaterial')] 
print(colNames)
scenariolist['@value|CumulativeEOLMaterial|Module#MetricTonnes'] = scenariolist[colNames].sum(axis=1)
foo = scenariolist[['@states', '@scenario|Module Composition Scenario', '@timeseries|Year', 
                    '@value|NewInstalledCapacity|PV#MW', '@value|CumulativeNewInstalledCapacity|PV#MW', 
                    '@value|InstalledCapacity|PV#MW',
                   '@value|CumulativeEOLMaterial|Module#MetricTonnes']]
foo.to_csv(csvtitle, index=False)
foo.head(5)

Index(['@value|CumulativeEOLMaterial|Glass#MetricTonnes',
       '@value|CumulativeEOLMaterial|Aluminium_frames#MetricTonnes',
       '@value|CumulativeEOLMaterial|Silver#MetricTonnes',
       '@value|CumulativeEOLMaterial|Silicon#MetricTonnes',
       '@value|CumulativeEOLMaterial|Copper#MetricTonnes'],
      dtype='object')


,@states,@scenario|Module Composition Scenario,@timeseries|Year,@value|NewInstalledCapacity|PV#MW,@value|CumulativeNewInstalledCapacity|PV#MW,@value|InstalledCapacity|PV#MW,@value|CumulativeEOLMaterial|Module#MetricTonnes
0,AL,Today,2018,205.018300,205.018300,206.132530,0.000000e+00
1,AL,Today,2019,2.210001,207.228300,207.239811,2.041243e-07
2,AL,Today,2020,2.210001,209.438301,208.834158,3.252767e-05
3,AL,Today,2021,4.017780,213.456081,212.240403,6.582875e-04
4,AL,Today,2022,4.017780,217.473861,215.628637,5.782718e-03
